In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

<body lang=ES-MX style='tab-interval:35.4pt'>

<div class=WordSection1>

<h3 align=center style='text-align:center'><b style='mso-bidi-font-weight:normal'><span
style='font-size:14.0pt;line-height:107%'>NOTEBOOK PARA OBTENER INFORMACION
SOBRE LAS INSTANCIAS QUE CORREN EN UN TITULO, SI ESTA “FUNCIONANDO” Y SU IP<o:p></o:p></span></b></h3>

<p class=MsoNormal><o:p>&nbsp;</o:p></p>

<p class=MsoNormal><span lang=EN-US style='mso-ansi-language:EN-US'>Options:<o:p></o:p></span></p>

<p class=MsoNormal><span lang=EN-US style='mso-ansi-language:EN-US'><span
style='mso-spacerun:yes'>    </span>-a Shows detailed info about the service
name including <span class=SpellE>instancenames</span>, <span class=SpellE>ip</span>,
hostname, depot,<br>
<span style='mso-spacerun:yes'>        </span><span
style='mso-spacerun:yes'> </span>CL, datacenter, <span class=SpellE>JiraOtName</span>,
Project Title, Build Location, etc.<o:p></o:p></span></p>

<p class=MsoNormal><span lang=EN-US style='mso-ansi-language:EN-US'><span
style='mso-spacerun:yes'>    </span>-d <span class=GramE>Shows</span> the
dedicated servers of the title<o:p></o:p></span></p>

<p class=MsoNormal><span lang=EN-US style='mso-ansi-language:EN-US'><span
style='mso-spacerun:yes'>    </span>-b <span class=GramE>Shows</span> the
databases boxes working with the title<o:p></o:p></span></p>

<p class=MsoNormal><span lang=EN-US style='mso-ansi-language:EN-US'><span
style='mso-spacerun:yes'>    </span>-r <span class=GramE>Shows</span> the <span
class=SpellE>redir</span> instances running with the title<o:p></o:p></span></p>

</div>

</body>

In [2]:
import sys
import json
import urllib3
import urllib2
import json
import sys
import csv
import xmltodict
import os
import socket
import struct
import pprint
import ipywidgets as wg
from IPython.display import display
#from nagios import (
#    plugin_exit, STATUS_CRIT, STATUS_OK, STATUS_WARN, STATUS_UNK
#)


ENVIRONMENTS = {'prod': 'http://tools.internal.gosredirector.ea.com:42125',
                'cert': 'http://internal.gosredirector.scert.ea.com:42125',
                'dev': 'http://internal.gosredirector.online.ea.com:42125',
                'test': 'http://internal.gosredirector.stest.ea.com:42125'}
SEARCH = '/redirector/getServerList?name='



def getInfo(service,env):
    try:
        data = urllib2.urlopen('http://torch.gameservices.ea.com/metadata/titleData.json')
    except:
        data = urllib2.urlopen('http://gosprodbeapp0008.abn-sjc.ea.com/portal/titleData.json')

    salida = json.load(data)
    servicess = []
    listbz = []
    listdb = []
    listds = []
    listrd = []
    listpt = []
    listjn = []
    listdc = []
    for product in salida:
        for blaze in product['serviceDescriptions']:
            if service == blaze['serviceName'] and env == blaze['environment']:
                try:
                    listbz = [x for x in blaze['appHosts']]
                except:
                    pass

                try:
                    listdb = [x[:-5] for x in blaze['databaseInstances']]
                except:
                    pass

                try:
                    listds = [x[:-6] for x in blaze['servulatorInstances']]
                except:
                    pass

                try:
                    listrd = [x for x in blaze['redisClusters']['main']]
                except:
                    pass

                try:
                    listpt.append(blaze['mdmProjectTitle'])
                except:
                    pass


                try:
                    listjn.append(blaze['jiraOTName'])
                except:
                    pass

                try:
                    listdc.append(blaze['dataCenter'])
                except:
                    pass



    return sorted(listbz), sorted(listdb), sorted(listds), sorted(listrd), sorted(servicess), sorted(listpt), sorted(listjn), sorted(listdc)


			
def int2ip(addr):
    return socket.inet_ntoa(struct.pack("!I", addr))


def searchinchredirector(schurl, sername, titledata):
    """
    Searching and getting the xml of redirector, analysing the xml file converting
    to a dictionary.
    This function checks if the service name, ip and instance exists in redirector
    xml file. If the instance exist it validates if is working fine or if is stuck
    (inservice 0 or 1)
    """


    try:
        to_unicode = unicode
    except NameError:
        to_unicode = str

    TITLESALERTS = {sername:schurl}
    con = 0
    nodata = 0
    instancesdct = {}
    instotal = []
    print "Processing ....."
    for x in TITLESALERTS.values():
        lstinstances2 = []
        cl = []
        depot = []
        buildlocation = []
        newList = []
        lstinstancesauxmasters = []
        lstinstancescoreslaves = []
        lstinstancesauxslaves = []
        titledictv2 = {}
        http2v2 = urllib3.PoolManager(retries=False)
        titlexmlv2 = http2v2.request('GET', x, timeout=3)
        titledictv2 = xmltodict.parse(titlexmlv2.data)
        if titledictv2['serverlist'] == None:
            print("Blaze Version not supported / Servicename Not exists")
            sys.exit()
        cl.append(str(titledictv2['serverlist']['servers']['serverinfodata']['version']))
        depot.append(str(titledictv2['serverlist']['servers']['serverinfodata']['depotlocation']))
        buildlocation.append(str(titledictv2['serverlist']['servers']['serverinfodata']['buildlocation']))

        lstinstances2.append(str(titledictv2['serverlist']['servers']['serverinfodata']['masterinstance']['instancename']))
        lstinstances2.append(str(titledictv2['serverlist']['servers']['serverinfodata']['masterinstance']['inservice']))
        lstinstances2.append(socket.getfqdn(str(int2ip(int(titledictv2['serverlist']['servers']['serverinfodata']['masterinstance']['endpoints']['serverendpointinfo'][1]['addresses']['serveraddressinfo']['address']['valu']['ip'])))))
        lstinstances2.append(str(int2ip(int(titledictv2['serverlist']['servers']['serverinfodata']['masterinstance']['endpoints']['serverendpointinfo'][1]['addresses']['serveraddressinfo']['address']['valu']['ip']))))

        for i in cl:
            newList.append(i.split(" ")[1])
        for h in newList:
            parts = h.split(".")[0]
#        print(parts)
        if parts in ["3"]:
            print("Blaze Version 3")
        try:
            for y in titledictv2['serverlist']['servers']['serverinfodata']['auxmasters']['serverinstance']:
                lstinstancesauxmasters.append([str(y.items()[4][1]), str(y.items()[5][1]), socket.getfqdn(str(int2ip(int(y.items()[1][1]['serverendpointinfo'][1]['addresses']['serveraddressinfo']['address']['valu']['ip'])))), int2ip(int(y.items()[1][1]['serverendpointinfo'][1]['addresses']['serveraddressinfo']['address']['valu']['ip']))])

#CORESLAVES INSTANCES
            for y in titledictv2['serverlist']['servers']['serverinfodata']['instances']['serverinstance']:
                s = []
                o = ['']
                o = y.items()[1]
                p = y.items()[0][1]
                s.append(p)
                t = ((o)[1]['serverendpointinfo'][1]['addresses']['serveraddressinfo'][0]['address']['valu']['ip'])
                j = str(int2ip(int(t)))
                g = socket.getfqdn(j)
                lstinstancescoreslaves.append([str(y.items()[4][1]), str(y.items()[5][1]), str(g),j])
#AUXSLAVES INSTANCES

            for y in titledictv2['serverlist']['servers']['serverinfodata']['auxslaves']['serverinstance']:
                lstinstancesauxslaves.append([str(y.items()[4][1]), str(y.items()[5][1]), socket.getfqdn(str(int2ip(int(y.items()[1][1]['serverendpointinfo'][1]['addresses']['serveraddressinfo']['address']['valu']['ip'])))), int2ip(int(y.items()[1][1]['serverendpointinfo'][1]['addresses']['serveraddressinfo']['address']['valu']['ip']))])
        except:
            try:
#CORESLAVES INSTANCES
                for y in titledictv2['serverlist']['servers']['serverinfodata']['instances']['serverinstance']:
                    s = []
                    o = ['']
                    o = y.items()[1]
                    p = y.items()[0][1]
                    s.append(p)
                    t = ((o)[1]['serverendpointinfo'][1]['addresses']['serveraddressinfo'][0]['address']['valu']['ip'])
                    j = str(int2ip(int(t)))
                    g = socket.getfqdn(j)

                    lstinstancescoreslaves.append([str(y.items()[4][1]), str(y.items()[5][1]), str(g),j])

            except:
                print"error en slaves"
        print("\n---------- DETAILED DATA ---------\n\n")
        print("Servicename: ", sername)
        print("Project Title: ", titledata[5])
        print("JiraOTName: ", titledata[6])
        print("Datacenter: ", titledata[7])
        print("CL: ", cl)
        print("Depot Location: ", depot)
        print("Build Location: ", buildlocation)
        print("\n---------- INSTANCES ---------\n ")
        pprint.pprint(lstinstances2)
        if lstinstancesauxmasters:
            for x in lstinstancesauxmasters:
                if x[1] == '0':
                    print("---------------------------------------------------------------------------")
                    pprint.pprint(x)
                    print("---------------------------------------------------------------------------")
                else:
                    pprint.pprint(x)
        else:
            print("There's not exists auxmaster instances")
        if lstinstancesauxslaves:
            for x in lstinstancesauxslaves:
                if x[1] == '0':
                    pprint.pprint("---------------------------------------------------------------------------")
                    pprint.pprint(x)
                    print("---------------------------------------------------------------------------")
                else:
                    pprint.pprint(x)
        else:
            print("There's not exists auxlave instances")
        if lstinstancescoreslaves:
            for x in lstinstancescoreslaves:
                if x[1] == '0':
                    print("---------------------------------------------------------------------------")
                    print(x)
                    print("---------------------------------------------------------------------------")
                else:
                    pprint.pprint(x)
        else:
            print("There's not exists auxlave instances")

        insnumber = 1+len(lstinstancesauxmasters)+len(lstinstancescoreslaves)+len(lstinstancesauxslaves)
        print("Running Instances: ",insnumber)
        print("\n")
        print("\n\n---------- RESUME DATA ----------\n\n")
        print("Hosts in: ", sername)
        hosts=[]
        hostssorted=[]
        hosts.append(lstinstances2[2])
        for x in lstinstancesauxmasters:
            hosts.append(x[2])
        for w in lstinstancesauxslaves:
            hosts.append(w[2])
        for z in lstinstancescoreslaves:
            hosts.append(z[2])
        hostssorted=sorted(hosts)
        for x in set(list(hostssorted)):
            print x
        print("Number of hosts used in this title: ",len(set(list(hostssorted))))

def from_string(ipv):
    "Convert dotted IPv4 address to integer."
    return reduce(lambda a, b: a<<8 | b, map(int, ipv.split(".")))

def on_button_clicked(b):
    os.system('clear')
    enviro = rdenviro.value    
    sname = name.value 
    options = opts.value
	
    envhost = ENVIRONMENTS.get(rdenviro.value, None)
    servicename = sname
    al = options
#    dirtycast = options.dirtycast or None
#    redis = options.redis or None
#    apphost = options.apphost or None
#    database = options.database or None
#    print(servicename)
#    print(envhost)
    searchurl = envhost+SEARCH+servicename+"$"
    x = getInfo(servicename,enviro)

    if al == "-a":
        print(searchurl)
        searchinchredirector(searchurl, servicename, x)
 
    if al == "-d":
        if x[2]:
            print("\n--------- DEDICATED SERVERS ---------\n")
            pprint.pprint(x[2])
        else:
            print("--------- THERE'S NOT EXIST DEDICATED SERVERS FOR THIS TITLE OWNED BY EA ---------\n")    
#print("Valor de la lista X")
#pprint.pprint (x)
    if al == "-p":
        if x[0]:
            print("\n--------- APP HOSTS ---------\n")
            pprint.pprint(x[0])
        else:
            print("--------- THERE'S NOT EXIST APP HOSTS FOR THIS TITLE ---------\n")
    if al == "-r":
        if x[3]:
            print("\n--------- REDIS INSTANCESS ---------\n")
            pprint.pprint(x[3])
        else:
            print("--------- THERE'S NOT EXIST REDIS INSTANCES FOR THIS TITLE  ---------\n")

    if all == "-b":
        if x[1]:
            print("\n--------- DATABASE BOXES ---------\n")
            pprint.pprint(x[1])
        else:
            print("--------- THERE'S NOT EXIST DATABASE BOXES FOR THIS TITLE  ---------\n")

#If not options has been choosen
#if (al == None and apphost == None and dirtycast == None and redis == None and apphost == None and database == None):
#    os.system("clear")
#    print("(----------- NOT VALID OPTIONS HAS BEEN CHOOSEN ----------)")
#    print("The sintaxis for this script is:")
#    print("instancesredir.py -e <environment> -s <servicename> -a 1 -d 1 -r 1 -p 1 -b 1")
#    print("Where -a 1 -d 1 -r 1 -p 1 and -b 1 are optional parameters")
#    print("-a 1 shows a detailed info for the service, instancenames, hosts, ips and inservice status for the instances")
#    print("-d 1 shows the dirtycasts boxes for the title")
#    print("-r 1 shows the redis instances for this title")
#    print("-b 1 shows the databases boxes for this title")
	

"""
Main function of the script.
"""
title = wg.Label(value='Write the Service Name to find:')
name = wg.Text(value='')
rdenviro = wg.RadioButtons(
    options=['prod', 'test', 'dev', 'cert'],
    value='prod',
    description='Environment: ',
    disabled=False
)
opts = wg.Select(
    options=['-a', '-b', '-d', '-r', '-p'],
    value='-a',
    # rows=10,
    description='Arguments:',
    disabled=False
)
btncontinue = wg.Button(
    description='Continue',
	disabled=False, 
	button_style='',
	tooltip='Click me to continue',
	icon='check'
	)

display(title,name,rdenviro,opts,btncontinue)

btncontinue.on_click(on_button_clicked)





Label(value=u'Write the Service Name to find:')

Text(value=u'')

RadioButtons(description=u'Environment: ', options=('prod', 'test', 'dev', 'cert'), value='prod')

Select(description=u'Arguments:', options=('-a', '-b', '-d', '-r', '-p'), value='-a')

Button(description=u'Continue', icon=u'check', style=ButtonStyle(), tooltip=u'Click me to continue')


--------- DEDICATED SERVERS ---------

[u'gosprodfeapp528.bio-dub.ea.com',
 u'gosprodfeapp557.bio-iad.ea.com',
 u'gosprodfeapp568.bio-sjc.ea.com']
http://tools.internal.gosredirector.ea.com:42125/redirector/getServerList?name=fifa-2016-ps4$
Processing .....

---------- DETAILED DATA ---------


('Servicename: ', u'fifa-2016-ps4')
('Project Title: ', [u'FIFA 16 GEN4'])
('JiraOTName: ', [u'FIFA 2016 Playstation 4'])
('Datacenter: ', [u'rs-lhr'])
('CL: ', ['Blaze 15.1.1.0.2 (CL# 1290633)'])
('Depot Location: ', ['gds-eadp-p4.rws.ad.ea.com:1777://gosdev/games/FIFA/2016/Gen4/prod/blazeserver/15.x'])
('Build Location: ', ['fifa@gosrtbprapp0002.bio-sjc.ea.com:/opt/home/fifa/p4/gosdev/games/FIFA/2016/Gen4/prod/blazeserver/15.x'])

---------- INSTANCES ---------
 
['configMaster0', '1', '10.99.52.20', '10.99.52.20']
['coreNSMaster1', '1', '10.99.52.20', '10.99.52.20']
['coreMaster2', '1', '10.99.52.20', '10.99.52.20']
['coreMaster3', '1', '10.99.52.20', '10.99.52.20']
['osdkAuxMaster4', '1', '